In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import joblib
from datetime import datetime
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# ==========================================
# ETAP 1: ZBIERANIE DANYCH (WGRYWANIE PLIKU)
# ==========================================
print("=== ETAP 1: WCZYTYWANIE DANYCH ===")
print("Proszę wybrać plik CSV z historią pomiarów (np. health_measurements.csv):")

uploaded = files.upload()

if not uploaded:
    print("Nie wybrano pliku!")
else:
    # Pobranie nazwy pierwszego pliku
    filename = next(iter(uploaded))

    # Wczytanie danych do DataFrame
    df = pd.read_csv(io.BytesIO(uploaded[filename]))

    # Konwersja daty (obsługa różnych formatów)
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

    # Sprawdzenie czy kolumna user_id istnieje (wymóg Wariantu 1)
    if 'user_id' not in df.columns:
        print("UWAGA: Brak kolumny 'user_id'. Tworzę domyślnego użytkownika 'user_1'.")
        df['user_id'] = 'user_1'

    # Konwersja user_id na string dla spójności
    df['user_id'] = df['user_id'].astype(str)

    print(f"\nWczytano {len(df)} rekordów.")
    print("Dostępni użytkownicy:", df['user_id'].unique())
    print("-" * 50)

    # ==========================================
    # INTERFEJS UŻYTKOWNIKA (SYMULACJA)
    # ==========================================
    print("\n=== KONFIGURACJA ANALIZY (WARIANT 1) ===")

    # Wybór użytkownika
    selected_user = input(f"Podaj ID użytkownika do analizy (dostępne: {list(df['user_id'].unique())}): ").strip()

    # Filtrowanie dat (opcjonalne)
    print("\nFiltrowanie dat (format YYYY-MM-DD). Pozostaw puste, aby pominąć.")
    start_date_str = input("Data początkowa: ").strip()
    end_date_str = input("Data końcowa: ").strip()

    # Logika filtrowania
    mask = (df['user_id'] == selected_user)
    if start_date_str:
        mask &= (df['timestamp'] >= pd.to_datetime(start_date_str))
    if end_date_str:
        mask &= (df['timestamp'] <= pd.to_datetime(end_date_str))

    user_df = df[mask].copy()

    if len(user_df) == 0:
        print(f"\nBrak danych dla użytkownika {selected_user} w zadanym okresie!")
    else:
        print(f"\nZnaleziono {len(user_df)} pomiarów dla użytkownika {selected_user}.")

        # ==========================================
        # ETAP 2: ANALIZA I WIZUALIZACJA DANYCH
        # ==========================================
        print(f"\n=== ETAP 2: ANALIZA DANYCH: {selected_user} ===")

        # Statystyki opisowe
        print("Statystyki podstawowe:")
        print(user_df[["age", "bmi", "glucose", "systolic_bp", "diastolic_bp"]].describe())

        # Wykres trendów
        if len(user_df) > 1:
            print("\nGenerowanie wykresu trendów...")
            plt.figure(figsize=(10, 5))
            plt.plot(user_df['timestamp'], user_df['systolic_bp'], marker='o', label='Ciśnienie skurczowe (SBP)')
            plt.plot(user_df['timestamp'], user_df['diastolic_bp'], marker='o', label='Ciśnienie rozkurczowe (DBP)')
            plt.axhline(y=140, color='r', linestyle='--', alpha=0.5, label='Próg nadciśnienia (140)')
            plt.title(f"Historia ciśnienia krwi: {selected_user}")
            plt.xlabel("Data")
            plt.ylabel("mmHg")
            plt.legend()
            plt.grid(True)
            plt.show()
        else:
            print("Za mało danych do wygenerowania wykresu trendu.")

        # ==========================================
        # ETAP 3: MODEL UCZENIA MASZYNOWEGO
        # ==========================================
        print("\n=== ETAP 3: TRENOWANIE MODELU ===")

        # Przygotowanie danych treningowych
        # Etykieta demonstracyjna: 1 jeśli SBP >= 140 lub DBP >= 90 (nadciśnienie)
        # UWAGA: W rzeczywistości etykieta powinna pochodzić z diagnozy lekarskiej!

        # Decyzja o strategii trenowania (Wariant 1)
        MIN_SAMPLES_FOR_PERSONAL_MODEL = 20

        if len(user_df) >= MIN_SAMPLES_FOR_PERSONAL_MODEL:
            print(f"Liczba pomiarów ({len(user_df)}) wystarczająca. Trenuję MODEL PERSONALNY dla {selected_user}.")
            train_df = user_df
        else:
            print(f"Za mało pomiarów użytkownika ({len(user_df)} < 20). Trenuję MODEL GLOBALNY na wszystkich danych.")
            train_df = df

        # Tworzenie etykiet (target)
        y = ((train_df['systolic_bp'] >= 140) | (train_df['diastolic_bp'] >= 90)).astype(int)
        X = train_df[['age', 'bmi', 'glucose', 'systolic_bp', 'diastolic_bp']]

        # Jeśli w danych jest tylko jedna klasa (np. wszyscy zdrowi), model nie zadziała
        if len(np.unique(y)) < 2:
            print("UWAGA: Dane treningowe zawierają tylko jedną klasę (np. same wyniki w normie).")
            print("Dodaję sztuczne próbki skrajne, aby umożliwić trening (cel demonstracyjny).")
            # Dodanie sztucznych rekordów dla zbalansowania (tylko demo!)
            X = pd.concat([X, pd.DataFrame({'age':[50,50], 'bmi':[25,30], 'glucose':[100,100], 'systolic_bp':[120,160], 'diastolic_bp':[80,100]})], ignore_index=True)
            y = pd.concat([y, pd.Series([0, 1])], ignore_index=True)

        # Pipeline preprocessingu
        numeric_features = ['age', 'bmi', 'glucose', 'systolic_bp', 'diastolic_bp']
        numeric_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, numeric_features)
            ])

        # Model
        clf = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', LogisticRegression())])

        # Trenowanie
        clf.fit(X, y)
        print("Model wytrenowany pomyślnie.")

        # ==========================================
        # ETAP 4: INTEGRACJA (PREDYKCJA)
        # ==========================================
        print(f"\n=== ETAP 4: PREDYKCJA DLA UŻYTKOWNIKA {selected_user} ===")
        print("Wprowadź nowe parametry do analizy:")

        try:
            # Pobranie danych od użytkownika (symulacja formularza aplikacji)
            new_age = float(input(f"Wiek (domyślnie ostatni: {user_df['age'].iloc[-1]}): ") or user_df['age'].iloc[-1])
            new_bmi = float(input("BMI: "))
            new_glucose = float(input("Glukoza: "))
            new_sbp = float(input("Ciśnienie skurczowe (SBP): "))
            new_dbp = float(input("Ciśnienie rozkurczowe (DBP): "))

            # Stworzenie ramki danych dla jednego rekordu
            X_new = pd.DataFrame({
                'age': [new_age],
                'bmi': [new_bmi],
                'glucose': [new_glucose],
                'systolic_bp': [new_sbp],
                'diastolic_bp': [new_dbp]
            })

            # Predykcja
            prediction = clf.predict(X_new)[0]
            probability = clf.predict_proba(X_new)[0][1]

            print("\n--- WYNIK ANALIZY ---")
            print(f"Prawdopodobieństwo ryzyka: {probability:.2%}")

            if prediction == 1:
                print("🔴 WYNIK: WYSOKIE RYZYKO (Zalecana konsultacja)")
            else:
                print("🟢 WYNIK: W NORMIE")

            # Logika "MVP": Opcja zapisu nowego pomiaru
            save_choice = input("\nCzy zapisać ten pomiar do historii? (t/n): ")
            if save_choice.lower() == 't':
                new_record = X_new.copy()
                new_record['user_id'] = selected_user
                new_record['timestamp'] = datetime.now()
                # W praktyce tutaj zapisalibyśmy do pliku CSV
                print("Pomiar dodano do lokalnej sesji (pamiętaj o pobraniu nowego pliku CSV, jeśli chcesz zachować zmiany).")

        except ValueError:
            print("Błąd wprowadzania danych. Proszę podawać liczby.")

=== ETAP 1: WCZYTYWANIE DANYCH ===
Proszę wybrać plik CSV z historią pomiarów (np. health_measurements.csv):


Saving health_measurements.csv to health_measurements.csv

Wczytano 5 rekordów.
Dostępni użytkownicy: ['1' '2' 'user_1' '3' '4']
--------------------------------------------------

=== KONFIGURACJA ANALIZY (WARIANT 1) ===
Podaj ID użytkownika do analizy (dostępne: ['1', '2', 'user_1', '3', '4']): 1

Filtrowanie dat (format YYYY-MM-DD). Pozostaw puste, aby pominąć.
Data początkowa: 1980-01-01
Data końcowa: 2026-02-06

Znaleziono 1 pomiarów dla użytkownika 1.

=== ETAP 2: ANALIZA DANYCH: 1 ===
Statystyki podstawowe:
        age   bmi  glucose  systolic_bp  diastolic_bp
count   1.0   1.0      1.0          1.0           1.0
mean   40.0  24.0     95.0        120.0          80.0
std     NaN   NaN      NaN          NaN           NaN
min    40.0  24.0     95.0        120.0          80.0
25%    40.0  24.0     95.0        120.0          80.0
50%    40.0  24.0     95.0        120.0          80.0
75%    40.0  24.0     95.0        120.0          80.0
max    40.0  24.0     95.0        120.0         